In [1]:
import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import matplotlib.pyplot as plt

import torch

from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import MAE, SMAPE, MultivariateNormalDistributionLoss

In [13]:
import numpy as np
import pandas as pd

test_data = pd.DataFrame(
    dict(
        value=np.random.rand(30) - 0.5,
        group=np.repeat(np.arange(3), 10),
        time_idx=np.tile(np.arange(10), 3),
    )
)
test_data[:11]

,value,group,time_idx
0,0.245166,0,0
1,0.261625,0,1
2,0.437428,0,2
3,0.244198,0,3
4,-0.259803,0,4
5,0.048446,0,5
6,0.394582,0,6
7,-0.349861,0,7
8,-0.098288,0,8
9,-0.044223,0,9


In [83]:
from pytorch_forecasting import TimeSeriesDataSet

# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    test_data,
    group_ids=["group"],
    target="value",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=2,
    max_prediction_length=2,
    time_varying_unknown_reals=["value"],
    target_normalizer=None,
)

# convert the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=4)

# and load the first batch
x, y = next(iter(dataloader))
print("x =", x)
print("\ny =", y)

x = {'encoder_cat': tensor([], size=(4, 5, 0), dtype=torch.int64), 'encoder_cont': tensor([[[ 0.1852],
         [ 0.1234],
         [-0.1009],
         [ 0.1711],
         [-0.1327]],

        [[-0.2827],
         [ 0.1852],
         [ 0.1234],
         [-0.1009],
         [ 0.1711]],

        [[ 0.4374],
         [ 0.2442],
         [-0.2598],
         [ 0.0484],
         [ 0.3946]],

        [[-0.4586],
         [ 0.4038],
         [ 0.1109],
         [ 0.0080],
         [-0.4632]]]), 'encoder_target': tensor([[ 0.1852,  0.1234, -0.1009,  0.1711, -0.1327],
        [-0.2827,  0.1852,  0.1234, -0.1009,  0.1711],
        [ 0.4374,  0.2442, -0.2598,  0.0484,  0.3946],
        [-0.4586,  0.4038,  0.1109,  0.0080, -0.4632]]), 'encoder_lengths': tensor([5, 5, 5, 5]), 'decoder_cat': tensor([], size=(4, 2, 0), dtype=torch.int64), 'decoder_cont': tensor([[[-0.1453],
         [-0.1180]],

        [[-0.1327],
         [-0.1453]],

        [[-0.3499],
         [-0.0983]],

        [[ 0.1585],
   

In [84]:
x.keys()

dict_keys(['encoder_cat', 'encoder_cont', 'encoder_target', 'encoder_lengths', 'decoder_cat', 'decoder_cont', 'decoder_target', 'decoder_lengths', 'decoder_time_idx', 'groups', 'target_scale'])

In [85]:
print(x['encoder_cat'])
print( x['encoder_cont'].shape )
print(x['encoder_target'].shape)
print(x['encoder_lengths'])
print("")

print(x['decoder_cat'])
print(x['decoder_cont'].shape)
print(x['decoder_target'].shape)
print(x['decoder_lengths'])
print(x['decoder_time_idx'])
print("")

print(x['groups'])
print(x['target_scale'])

tensor([], size=(4, 5, 0), dtype=torch.int64)
torch.Size([4, 5, 1])
torch.Size([4, 5])
tensor([5, 5, 5, 5])

tensor([], size=(4, 2, 0), dtype=torch.int64)
torch.Size([4, 2, 1])
torch.Size([4, 2])
tensor([2, 2, 2, 2])
tensor([[8, 9],
        [7, 8],
        [7, 8],
        [5, 6]])

tensor([[1],
        [1],
        [0],
        [2]])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])


In [86]:
test_data[:10]

,value,group,time_idx
0,0.245166,0,0
1,0.261625,0,1
2,0.437428,0,2
3,0.244198,0,3
4,-0.259803,0,4
5,0.048446,0,5
6,0.394582,0,6
7,-0.349861,0,7
8,-0.098288,0,8
9,-0.044223,0,9


In [87]:
dataset.data

{'reals': tensor([[ 0.2452],
         [ 0.2616],
         [ 0.4374],
         [ 0.2442],
         [-0.2598],
         [ 0.0484],
         [ 0.3946],
         [-0.3499],
         [-0.0983],
         [-0.0442],
         [ 0.2401],
         [ 0.3211],
         [-0.2827],
         [ 0.1852],
         [ 0.1234],
         [-0.1009],
         [ 0.1711],
         [-0.1327],
         [-0.1453],
         [-0.1180],
         [-0.4586],
         [ 0.4038],
         [ 0.1109],
         [ 0.0080],
         [-0.4632],
         [ 0.1585],
         [ 0.2708],
         [ 0.0527],
         [ 0.3262],
         [ 0.3569]]),
 'categoricals': tensor([], size=(30, 0), dtype=torch.int64),
 'groups': tensor([[0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [2],
         [2],
         

In [88]:
print(x['encoder_cont'].squeeze())

tensor([[ 0.1852,  0.1234, -0.1009,  0.1711, -0.1327],
        [-0.2827,  0.1852,  0.1234, -0.1009,  0.1711],
        [ 0.4374,  0.2442, -0.2598,  0.0484,  0.3946],
        [-0.4586,  0.4038,  0.1109,  0.0080, -0.4632]])


In [89]:
print(x['decoder_cont'].squeeze())

tensor([[-0.1453, -0.1180],
        [-0.1327, -0.1453],
        [-0.3499, -0.0983],
        [ 0.1585,  0.2708]])


In [90]:
print(y)

(tensor([[-0.1453, -0.1180],
        [-0.1327, -0.1453],
        [-0.3499, -0.0983],
        [ 0.1585,  0.2708]]), None)
